In [104]:
import pandas as pd
import numpy as np

In [105]:
df_athletes = pd.read_json('/kaggle/input/olympic/olympic_athletes.json')
df_athletes['athlete_medals'] = df_athletes['athlete_medals'].replace(r'\n', '', regex=True)
df_athletes.head()

,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,athlete_medals,bio
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000.0,None,None
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995.0,None,None
2,https://olympics.com/en/athletes/dylan-walczyk,Dylan WALCZYK,1,Beijing 2022,1993.0,None,None
3,https://olympics.com/en/athletes/olli-penttala,Olli PENTTALA,1,Beijing 2022,1995.0,None,None
4,https://olympics.com/en/athletes/reikherd,Dmitriy REIKHERD,1,Beijing 2022,1989.0,None,None


In [106]:
df_hosts = pd.read_xml('/kaggle/input/olympic/olympic_hosts.xml')
df_hosts.set_index('index', inplace=True)
df_hosts.head()

,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
index,,,,,,,
0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
2,pyeongchang-2018,2018-02-25T08:00:00Z,2018-02-08T23:00:00Z,Republic of Korea,PyeongChang 2018,Winter,2018
3,rio-2016,2016-08-21T21:00:00Z,2016-08-05T12:00:00Z,Brazil,Rio 2016,Summer,2016
4,sochi-2014,2014-02-23T16:00:00Z,2014-02-07T04:00:00Z,Russian Federation,Sochi 2014,Winter,2014


In [107]:
df_results = pd.read_html('/kaggle/input/olympic/olympic_results.html',index_col=0)[0]
df_results.head(20)

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,NaN,NaN,NaN,NaN
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,NaN,NaN,NaN,NaN
2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",False,3,Sweden,SE,SWE,NaN,NaN,NaN,NaN
3,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Jennifer DODDS', 'https://olympics.com/en/a...",False,4,Great Britain,GB,GBR,NaN,NaN,NaN,NaN
4,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",False,5,Canada,CA,CAN,NaN,NaN,NaN,NaN
5,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Zuzana HAJKOVA', 'https://olympics.com/en/a...",False,6,Czech Republic,CZ,CZE,NaN,NaN,NaN,NaN
6,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Jenny PERRET', 'https://olympics.com/en/ath...",False,7,Switzerland,CH,SUI,NaN,NaN,NaN,NaN
7,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Vicky PERSINGER', 'https://olympics.com/en/...",False,8,United States of America,US,USA,NaN,NaN,NaN,NaN
8,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Suyuan FAN', 'https://olympics.com/en/athle...",False,9,People's Republic of China,CN,CHN,NaN,NaN,NaN,NaN
9,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Tahli GILL', 'https://olympics.com/en/athle...",False,10,Australia,AU,AUS,NaN,NaN,NaN,NaN


In [108]:
print(df_results['athlete_url'].isna().sum())
print(df_results['athlete_full_name'].isna().sum())
print(df_results['athletes'].isna().sum())
print(df_results['value_unit'].isna().sum())
print(df_results['value_type'].isna().sum())

32813
21158
154828
84158
72755


In [109]:
len(df_results)

162804

In [110]:
df_results['value_type'].value_counts().head(10)

value_type
TIME           43774
POINTS         29193
IRM             9079
WEIGHT          3439
DISTANCE        3003
RANK            1251
STROKES          238
SCORE             45
NO_TIME            9
SC_REST$IRM        7
Name: count, dtype: int64

In [111]:
def format_athletes_unique_format(row):
    if pd.notna(row['athletes']):
        return row['athletes']
    elif pd.notna(row['athlete_full_name']):
        return [(row['athlete_full_name'], row['athlete_url'] )]
df_results['athletes'] = df_results.apply(format_athletes_unique_format, axis=1)
df_results.head(50)

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,NaN,NaN,NaN,NaN
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,NaN,NaN,NaN,NaN
2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",False,3,Sweden,SE,SWE,NaN,NaN,NaN,NaN
3,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Jennifer DODDS', 'https://olympics.com/en/a...",False,4,Great Britain,GB,GBR,NaN,NaN,NaN,NaN
4,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",False,5,Canada,CA,CAN,NaN,NaN,NaN,NaN
5,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Zuzana HAJKOVA', 'https://olympics.com/en/a...",False,6,Czech Republic,CZ,CZE,NaN,NaN,NaN,NaN
6,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Jenny PERRET', 'https://olympics.com/en/ath...",False,7,Switzerland,CH,SUI,NaN,NaN,NaN,NaN
7,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Vicky PERSINGER', 'https://olympics.com/en/...",False,8,United States of America,US,USA,NaN,NaN,NaN,NaN
8,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Suyuan FAN', 'https://olympics.com/en/athle...",False,9,People's Republic of China,CN,CHN,NaN,NaN,NaN,NaN
9,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Tahli GILL', 'https://olympics.com/en/athle...",False,10,Australia,AU,AUS,NaN,NaN,NaN,NaN


In [112]:
df_results.columns

Index(['discipline_title', 'event_title', 'slug_game', 'participant_type',
       'medal_type', 'athletes', 'rank_equal', 'rank_position', 'country_name',
       'country_code', 'country_3_letter_code', 'athlete_url',
       'athlete_full_name', 'value_unit', 'value_type'],
      dtype='object')

In [113]:
df_results = df_results.drop(['athlete_full_name', 'athlete_url'], axis=1)
df_results['athletes'].isna().sum() / df_results.shape[0] * 100

8.09685265718287

In [114]:
df_results.tail(10)

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,value_unit,value_type
162794,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,NaN,"[(Christopher E. W. MACKINTOSH, https://olympi...",NaN,DNS,Great Britain,GB,GBR,NaN,IRM
162795,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,NaN,"[(Istvan DEVAN, https://olympics.com/en/athlet...",NaN,DNS,Hungary,HU,HUN,NaN,IRM
162796,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,NaN,"[(AladÃ¡r HÃ¡berl, nan)]",NaN,DNS,Hungary,HU,HUN,NaN,IRM
162797,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,NaN,"[(Ivo Demenego, nan)]",NaN,DNS,Italy,IT,ITA,NaN,IRM
162798,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,NaN,"[(Pio Imboden, nan)]",NaN,DNS,Italy,IT,ITA,NaN,IRM
162799,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,NaN,"[(Franciszek BUJAK, https://olympics.com/en/at...",NaN,DNS,Poland,PL,POL,NaN,IRM
162800,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,NaN,"[(Henryk MÃ¼ckenbrunn, nan)]",NaN,DNS,Poland,PL,POL,NaN,IRM
162801,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,NaN,"[(Milda Prokopec, nan)]",NaN,DNS,Czechoslovakia,CSHH,TCH,NaN,IRM
162802,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,NaN,"[(Sigurd Overby, nan)]",NaN,DNS,United States of America,US,USA,NaN,IRM
162803,Gymnastics Artistic,"26th Federal Festival, Society and Association...",paris-1900,GameTeam,NaN,None,NaN,1,France,FR,FRA,142 pts,POINTS


In [115]:
df_medals = pd.read_excel('/kaggle/input/olympic/olympic_medals.xlsx',index_col=0)
df_medals.head(10)

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,Italy,IT,ITA
1,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,Italy,IT,ITA
2,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/kristin-skaslien,Kristin SKASLIEN,Norway,NO,NOR
3,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/magnus-nedreg...,Magnus NEDREGOTTEN,Norway,NO,NOR
4,Curling,beijing-2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,https://olympics.com/en/athletes/almida-de-val,Almida DE VAL,Sweden,SE,SWE
5,Curling,beijing-2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,https://olympics.com/en/athletes/oskar-eriksson,Oskar ERIKSSON,Sweden,SE,SWE
6,Curling,beijing-2022,Women,Women,GOLD,GameTeam,Great Britain,NaN,NaN,Great Britain,GB,GBR
7,Curling,beijing-2022,Women,Women,SILVER,GameTeam,Japan,NaN,NaN,Japan,JP,JPN
8,Curling,beijing-2022,Women,Women,BRONZE,GameTeam,Sweden,NaN,NaN,Sweden,SE,SWE
9,Curling,beijing-2022,Men,Men,GOLD,GameTeam,Sweden,NaN,NaN,Sweden,SE,SWE


In [116]:
df_medals.columns

Index(['discipline_title', 'slug_game', 'event_title', 'event_gender',
       'medal_type', 'participant_type', 'participant_title', 'athlete_url',
       'athlete_full_name', 'country_name', 'country_code',
       'country_3_letter_code'],
      dtype='object')

In [117]:
def get_type_by_event_gender(row):
    if "women" in row['event_title'].lower() or "ladi" in row['event_title'].lower():
        return 'Women'
    elif "men" in row['event_title'].lower():
        return 'Men'
    elif "mixed" in row['event_title'].lower():
        return 'Mixed'
    else:
        return 'Men'
    
def format_unit(unit_type, unit_value):
    if (unit_type == 'DISTANCE' and isinstance(unit_value, str) and 'w' in unit_value):
        return unit_value.replace('w', '')
    elif unit_type == 'TIME' and isinstance(unit_value, str):
        try:
            total_milliseconds = int(unit_value)
            milliseconds = total_milliseconds % 1000
            total_seconds = total_milliseconds // 1000
            seconds = total_seconds % 60
            total_minutes = total_seconds // 60
            minutes = total_minutes % 60
            hours = total_minutes // 60
            return f"{hours:02}:{minutes:02}:{seconds:02}:{milliseconds:03}"
        except:
            return unit_value
        
    return unit_value
        
    
df_results['event_gender'] = df_results.apply(get_type_by_event_gender, axis=1)
df_results['value_unit'] = df_results.apply(lambda row: format_unit(row['value_type'], row['value_unit']), axis=1)

In [118]:
df_hosts.to_csv('/kaggle/working/hosts.csv', index=False, encoding='utf-8')

In [119]:
df_results.to_csv('/kaggle/working/results_jo.csv', index=False, encoding='utf-8')

In [120]:
df_athletes.to_csv('/kaggle/working/athletes.csv', index=False, encoding='utf-8')